## Definition

### Problem Statement  

The goal is to forecast the demand for bikes in dependency of weather conditions like outside temperature and calendric informations e.g. holidays. These information and the demand structure is provided in a set with two years of daily historic data.  
The demand is given as the total daily demand and as a split for registered users and casual users. To increase the quality of the prediction registered user demand and casual user demand will be predicted separately in step two.  
To make predictions machine learning is used to train regressors. Scikit-Learn recommends a support vector regressor (SVR) for this kind of problem and data amount. In addition a deep neuronal network (DNN) regressor is trained for comparison. To find the hyper-parameters for these regressors grid search and randomized search are utilized. Due to the small dataset cross validation is applied.    

> http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR  
> https://github.com/tensorflow/skflow/blob/master/g3doc/api_docs/python/estimators.md  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.RandomizedSearchCV.html

In [3]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar

from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from math import sqrt

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Analysis

In [6]:
# Fetching Dataset

bike_data = pd.read_csv("day.csv", header=0)

print("Data read successfully!")

Data read successfully!


In [7]:
bike_data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


### Data Exploration

In [8]:
# Extracting

feature_cols = bike_data.columns[:-3]  # all columns but last are features
target_col = bike_data.columns[-1]  # last column is the target

print ("Feature column(s):\n{}\n".format(feature_cols))
print ("Target column:\n{}".format(target_col))

Feature column(s):
Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed'],
      dtype='object')

Target column:
cnt


In [9]:
#### Function to Calculate Profit

In [10]:
def profit(y,y_cap):
    return 3 * np.minimum(y[::1], y_cap[::1]) - 2 * y_cap[::1]
    

#### Function to Convert from percentage to Actual Prediction

In [11]:
def convertToPrediction(data,percentage_predictions):
    demand = np.around(data + (np.multiply(data, percentage_predictions)/100))
    demand[demand <0 ] = 0
    return demand

##### RMSLE Scorer
One common way to evaluate regression model is through calculating MSE or RMSE. In this particular project , the metric to evaluate our model is Root Mean Square Logarithmic Error (RMSLE). RMSLE is particularly helpful when you want to penalize an under-predicted estimate greater than an over-predicted estimate.

In [53]:
def rmsle(y, y_):
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

### Base Model

#### For the base model the demand for today is the previous days demand. 

In [12]:
y_actual = bike_data[target_col][365:731]  # corresponding targets
y_actual = y_actual.reset_index(drop = True)

In [13]:
y_staged = y_actual.copy()

In [14]:
data = []
data.insert(0, bike_data[target_col][364])
data.insert(0, bike_data[target_col][363])

In [15]:
y_predicted_df = pd.concat([pd.DataFrame(data), y_staged], ignore_index=True)

In [16]:
y_predicted_df.drop(y_predicted_df.tail(2).index,inplace=True)

In [17]:
y_predicted = y_predicted_df[0]


##### Calculate Base Model Profit

In [18]:
print(profit(y_actual,y_predicted).sum())

1442972


#### Dataset with percentage change

In [ ]:
X_test = X_raw_test[cols].values.tolist()
y_test_df = X_raw_test[['target']]
y_test = y_test_df['target'].tolist()

In [19]:
data = pd.read_csv("processed_Data.csv", header=0)
data['instant'] = data['instant'] % 30
X_raw_train = data[0:359]
X_raw_test  = data[359:]

In [20]:
cols =[
       "season__1","season__2","season__3","season__4","season__5",
       "weathersit__1","weathersit__2","weathersit__3","weathersit__4","weathersit__5",
        "cnt__1",
        "atemp","hum","windspeed",
        "mnth","instant","holiday","weekday","workingday",
        "moving_avg_weekly_cnt"]     

In [21]:
X_train = X_raw_train[cols].values.tolist()
y_train_df = X_raw_train[['demand_pc_inc']]
y_train = y_train_df['demand_pc_inc'].tolist()

In [22]:
X_test = X_raw_test[cols].values.tolist()
y_test_df = X_raw_test[['demand_pc_inc']]
y_test = y_test_df['demand_pc_inc'].tolist()

In [23]:
data_cnt = data['cnt']

In [24]:
actual_predictions = data_cnt[359:].values

In [25]:
y_for_calculations = data_cnt[357:723].values

### Algorithms and Techniques

##### Ensemble Methods

Ensemble methods are meta-algorithms that combine several machine learning techniques into one predictive model in order to decrease variance (bagging), bias (boosting), or improve predictions (stacking). Ensemble methods can be divided into two groups: sequential ensemble methods where the base learners are generated sequentially (e.g. AdaBoost) and parallel ensemble methods where the base learners are generated in parallel (e.g. Random Forest). The basic motivation of sequential methods is to exploit the dependence between the base learners since the overall performance can be boosted by weighing previously mislabeled examples with higher weight. The basic motivation of parallel methods is to exploit independence between the base learners since the error can be reduced dramatically by averaging.

#### Boosting
Boosting refers to a family of algorithms that are able to convert weak learners to strong learners. The main principle of boosting is to fit a sequence of weak learners (models that are only slightly better than random guessing, such as small decision trees) to weighted versions of the data, where more weight is given to examples that were mis-classified by earlier rounds. The predictions are then combined through a weighted majority vote (classification) or a weighted sum (regression) to produce the final prediction. The principal difference between boosting and the committee methods such as bagging is that base learners are trained in sequence on a weighted version of the data.

###### Adaptive Boosting

The algorithm below describes the most widely used form of boosting algorithm called AdaBoost, which stands for adaptive boosting.

In [30]:
### ADA Boost Regressor

from sklearn.ensemble import AdaBoostRegressor
from sklearn import pipeline,metrics,grid_search

#regressor = RandomForestRegressor(random_state = 0, max_depth = 30, n_estimators = 500, max_features = 'log2')
regressor = AdaBoostRegressor()
estimator_ada = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator_ada.fit(X_train, y_train)
pred_ada = estimator_ada.predict(X_test)
model_predictions_ada = convertToPrediction(y_for_calculations,pred_ada)
print(profit(actual_predictions,model_predictions_ada).sum())

1508066.0


In [58]:
num_est = [1, 2, 3, 10,50,100,200,300,500]
learning_rate = [0.01,0.1,0.5,1]
loss = ['linear', 'square', 'exponential']
params_dict={'n_estimators':num_est,'learning_rate': learning_rate,'loss':loss}
clf_ada=GridSearchCV(estimator=AdaBoostRegressor(),param_grid=params_dict,scoring='neg_mean_squared_error')
clf_ada.fit(X_train,y_train)
pred_ada=clf_ada.predict(X_test)
model_predictions_ada = convertToPrediction(y_for_calculations,pred_ada)
print(profit(actual_predictions,model_predictions_ada).sum())

1456869.0


In [59]:
print("Best params: ", clf_ada.best_params_)

Best params:  {'learning_rate': 1, 'loss': 'linear', 'n_estimators': 2}


In [85]:
regressor = AdaBoostRegressor(n_estimators = 200, loss = 'square')
estimator_ada = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator_ada.fit(X_train, y_train)
pred_ada = estimator_ada.predict(X_test)
model_predictions_ada = convertToPrediction(y_for_calculations,pred_ada)
print(profit(actual_predictions,model_predictions_ada).sum())

1522876.0


In [55]:
print ("RMSLE Value: ",rmsle(actual_predictions,model_predictions_ada))

RMSLE Value:  0.47134455308457096


In [22]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [23]:
#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification
 

In [119]:
models=[RandomForestRegressor(),GradientBoostingRegressor(),AdaBoostRegressor(),BaggingRegressor(),SVR(),KNeighborsRegressor()]
model_names=['RandomForestRegressor','GradientBoostingRegressor','AdaBoostRegressor','BaggingRegressor','SVR','KNeighborsRegressor']
rmsle=[]
d={}
for model in range (len(models)):
    clf=models[model]
    print(model_names[model])
    clf.fit(X_train,y_train)
    test_pred=clf.predict(X_test)
    model_predictions = convertToPrediction(y_for_calculations,test_pred)
    print(profit(actual_predictions,model_predictions).sum())

RandomForestRegressor
1584083.0
GradientBoostingRegressor
1581123.0
AdaBoostRegressor
1509126.0
BaggingRegressor
1595757.0
SVR
1438256.0
KNeighborsRegressor
1445710.0


## Methodology

In [ ]:
#NOW LET'S Dig deeper into each of these ...

### Implementation

The regressors are trained using randomized search and cross-validation to identify the area of the best parameters. Then a grid search is used to tune parameter values of the regressor functions.

> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.RandomizedSearchCV.html

##### Random Forest Regressor

In [120]:

#for random forest regresion.
no_of_test=[500]
params_dict={'n_estimators':no_of_test,'n_jobs':[-1],'max_features':["auto",'sqrt','log2'],'max_depth':[10,20,30]}
clf_rf=GridSearchCV(estimator=RandomForestRegressor(),param_grid=params_dict,scoring='neg_mean_squared_error')
clf_rf.fit(X_train,y_train)
pred_rf=clf_rf.predict(X_test)
model_predictions_rf = convertToPrediction(y_for_calculations,pred_rf)
print(profit(actual_predictions,model_predictions_rf).sum())

1548673.0


In [121]:
print("Best params: ", clf_rf.best_params_)

Best params:  {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 500, 'n_jobs': -1}


In [122]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import pipeline,metrics,grid_search

#regressor = RandomForestRegressor(random_state = 0, max_depth = 30, n_estimators = 500, max_features = 'log2')
regressor = RandomForestRegressor()
estimator_rf = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator_rf.fit(X_train, y_train)
pred_rf = estimator_rf.predict(X_test)
model_predictions_rf = convertToPrediction(y_for_calculations,pred_rf)
print(profit(actual_predictions,model_predictions_rf).sum())

1598299.0


#### Gradient Boosting Regressor

In [123]:
#for Gradient Boosting regresion.
no_of_estimators=[100,200,300,400,500]
params_dict={'n_estimators':no_of_estimators,'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
             'max_features':["auto",'sqrt','log2'],'max_depth':[10,20,30,40,50]}
clf_gbr=GridSearchCV(estimator=GradientBoostingRegressor(),param_grid=params_dict,scoring='neg_mean_squared_error')
clf_gbr.fit(X_train,y_train)
pred=clf_gbr.predict(X_test)
model_predictions = convertToPrediction(y_for_calculations,pred)
print(profit(actual_predictions,model_predictions).sum())

1520967.0


In [30]:
print("Best params: ", clf_gbr.best_params_)

Best params:  {'learning_rate': 0.3, 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100}


In [125]:
from sklearn.ensemble import GradientBoostingRegressor

#gbr = GradientBoostingRegressor(n_estimators=500,max_features= 'log2', learning_rate=0.1, max_depth = 10)
gbr = GradientBoostingRegressor()

estimator_gbr = pipeline.Pipeline(steps = [       
    ('model_fitting', gbr)
    ]
)
estimator_gbr.fit(X_train, y_train)
pred_gbr = estimator_gbr.predict(X_test)
model_predictions_gbr = convertToPrediction(y_for_calculations,pred_gbr)
print(profit(actual_predictions,model_predictions_gbr).sum())

1581854.0


#### ADA Boost Regressor

In [129]:
### ADA Boost Regressor

from sklearn.ensemble import AdaBoostRegressor
from sklearn import pipeline,metrics,grid_search

#regressor = RandomForestRegressor(random_state = 0, max_depth = 30, n_estimators = 500, max_features = 'log2')
regressor = AdaBoostRegressor()
estimator_ada = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator_ada.fit(X_train, y_train)
pred_ada = estimator_ada.predict(X_test)
model_predictions_ada = convertToPrediction(y_for_calculations,pred_ada)
print(profit(actual_predictions,model_predictions_ada).sum())

1521528.0


#### Bagging Regressor

In [134]:
### Bagging Regressor

from sklearn.ensemble import BaggingRegressor
from sklearn import pipeline,metrics,grid_search

#regressor = RandomForestRegressor(random_state = 0, max_depth = 30, n_estimators = 500, max_features = 'log2')
regressor = BaggingRegressor()
estimator_bagging = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator_bagging.fit(X_train, y_train)
pred_bagging = estimator_bagging.predict(X_test)
model_predictions_bagging = convertToPrediction(y_for_calculations,pred_bagging)
print(profit(actual_predictions,model_predictions_bagging).sum())


1584347.0


#### KNN Regressor

In [135]:
### KNN Regressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import pipeline,metrics,grid_search


regressor = KNeighborsRegressor()
estimator_knn = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator_knn.fit(X_train, y_train)
pred_knn = estimator_knn.predict(X_test)
model_predictions_knn = convertToPrediction(y_for_calculations,pred_knn)
print(profit(actual_predictions,model_predictions_knn).sum())




1445710.0


#### SVM Regressor

In [136]:
# Training SVR
svr = SVR()
svr.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [137]:
# Validation SVR

pred_svr = svr.predict(X_test)
score_svr = r2_score(y_test, pred_svr)
rmse_svr = sqrt(mean_squared_error(y_test, pred_svr))

print("Score SVR: %f" % score_svr)
print("RMSE SVR: %f" % rmse_svr)

Score SVR: -0.003305
RMSE SVR: 1318.397271


In [138]:
# Tuning SVR with GridSearch

tuned_parameters = [{'C': [1000, 3000, 10000], 
                     'kernel': ['linear', 'rbf']}
                   ]

#svr_tuned = GridSearchCV(SVR (C=1), param_grid = tuned_parameters, scoring = 'mean_squared_error') #default 3-fold cross-validation, score method of the estimator
svr_tuned_GS = GridSearchCV(SVR (C=1), param_grid = tuned_parameters, scoring = 'r2', n_jobs=-1) #default 3-fold cross-validation, score method of the estimator

svr_tuned_GS.fit(X_train, y_train)

print (svr_tuned_GS)
print ('\n' "Best parameter from grid search: " + str(svr_tuned_GS.best_params_) +'\n')

GridSearchCV(cv=None, error_score='raise',
       estimator=SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'C': [1000, 3000, 10000], 'kernel': ['linear', 'rbf']}],
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

Best parameter from grid search: {'C': 1000, 'kernel': 'rbf'}



In [139]:
svr_tuned_pred_GS = svr_tuned_GS.predict(X_test)

score_svr_tuned_GS = r2_score(y_test, svr_tuned_pred_GS)
rmse_svr_tuned_GS = sqrt(mean_squared_error(y_test, svr_tuned_pred_GS))

print("SVR Results\n")

print("Score SVR: %f" % score_svr)
print("Score SVR tuned GS: %f" % score_svr_tuned_GS)

print("\nRMSE SVR: %f" % rmse_svr)
print("RMSE SVR tuned GS: %f" % rmse_svr_tuned_GS)

SVR Results

Score SVR: -0.003305
Score SVR tuned GS: -0.002542

RMSE SVR: 1318.397271
RMSE SVR tuned GS: 1317.895584


In [140]:
svr_tuned_pred_GS


##Profit Calculation for pct approach
model_predictions = convertToPrediction(y_for_calculations,svr_tuned_pred_GS)
print(profit(actual_predictions,model_predictions).sum())

#Profit is just 1.26million!!

1269779.0


In [141]:
# SVR tuned with RandomizesSearch
# may take a while!

# Parameters
param_dist = {  'C': sp_uniform (1000, 10000), 
                'kernel': ['rbf']
             }

n_iter_search = 1

# MSE optimized
#SVR_tuned_RS = RandomizedSearchCV(SVR (C=1), param_distributions = param_dist, scoring = 'mean_squared_error', n_iter=n_iter_search)

# R^2 optimized
SVR_tuned_RS = RandomizedSearchCV(SVR (C=1), param_distributions = param_dist, scoring = 'r2', n_iter=n_iter_search)

# Fit
SVR_tuned_RS.fit(X_train, y_train)

# Best score and corresponding parameters.
print('best CV score from grid search: {0:f}'.format(SVR_tuned_RS.best_score_))
print('corresponding parameters: {}'.format(SVR_tuned_RS.best_params_))

# Predict and score
predict = SVR_tuned_RS.predict(X_test)

score_svr_tuned_RS = r2_score(y_test, predict)
rmse_svr_tuned_RS = sqrt(mean_squared_error(y_test, predict))

best CV score from grid search: -0.034399
corresponding parameters: {'C': 4289.95475119251, 'kernel': 'rbf'}


In [142]:
print('SVR Results\n')

print("Score SVR: %f" % score_svr)
print("Score SVR tuned GS: %f" % score_svr_tuned_GS)
print("Score SVR tuned RS: %f" % score_svr_tuned_RS)

print("\nRMSE SVR: %f" % rmse_svr)
print("RMSE SVR tuned GS: %f" % rmse_svr_tuned_GS)
print("RMSE SVR tuned RS: %f" % rmse_svr_tuned_RS)

SVR Results

Score SVR: -0.003305
Score SVR tuned GS: -0.002542
Score SVR tuned RS: -0.002542

RMSE SVR: 1318.397271
RMSE SVR tuned GS: 1317.895584
RMSE SVR tuned RS: 1317.895584


In [143]:
##Profit Calculation for pct approach
model_predictions = convertToPrediction(y_for_calculations,predict)
print(profit(actual_predictions,model_predictions).sum())

1269779.0


### DNN Regressor

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
import logging
from concurrent.futures import ThreadPoolExecutor, wait
from time import time
from typing import List

In [ ]:
bike_model = MLPRegressor(hidden_layer_sizes=(5,),
                                       activation='relu',
                                       solver='adam',
                                       learning_rate='adaptive',
                                       max_iter=15000,
                                       learning_rate_init=0.01,
                                       alpha=0.01)

In [ ]:
start_time = int(time() * 1000)
bike_model.fit(X_train, y_train)
end_time = int(time() * 1000)
logging.debug('Finished training universal model')
logging.debug('Training took {} ms'.format(end_time - start_time)) 

In [ ]:
pred_dnn = bike_model.predict(X_test)

In [ ]:
model_predictions_dnn = convertToPrediction(y_for_calculations,pred_dnn)

In [144]:
print(profit(actual_predictions,model_predictions_dnn).sum())

NameError: name 'model_predictions_dnn' is not defined

#### Ensembles

In [ ]:
# Averageing best predictions from different regressors

In [145]:
w = [1, 1, 1, 1]  # weights
pred_agg = np.c_[pred_rf,pred_gbr,pred_ada,pred_bagging]
pred_avr = np.average(pred_agg, axis=1, weights=w)
model_predictions_avr = convertToPrediction(y_for_calculations,pred_avr)
print(profit(actual_predictions,model_predictions_avr).sum())

1606967.0


In [36]:
# Ensemble : Stacking

In [146]:
pred_train_rf = estimator_rf.predict(X_train)
pred_train_gbr = estimator_gbr.predict(X_train)
pred_train_ada = estimator_ada.predict(X_train)
pred_train_bagging = estimator_bagging.predict(X_train)
pred_train_knn = estimator_knn.predict(X_train)

In [147]:
stack = pd.DataFrame({'rf':pred_train_rf, 'gbr':pred_train_gbr, 'ada':pred_train_ada,
                      'bagging':pred_train_bagging,'true':y_train})

In [148]:
from tensorflow.contrib import learn
import tensorflow as tf

# Stacking multiple regressors using DNN
features_blend = learn.infer_real_valued_columns_from_input(stack[['rf','gbr','ada','bagging']])

# Optimizer algorithm
adam = tf.train.AdamOptimizer(learning_rate=0.001)

# Build multi-layer DNN for regression
blend_nn = learn.DNNRegressor(feature_columns=features_blend, hidden_units=[50, 20], 
                              optimizer=adam, activation_fn=tf.nn.leaky_relu)
#                               config=learn.estimators.RunConfig(num_cores=8))
# Fit DNN
blend_nn.fit(x=stack[['rf','gbr','ada','bagging']], y=stack['true'], steps=8000, batch_size=256)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11df38a58>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/qk/0vhw5p4n2610vyrs2zqjmj2h0000gn/T/tmp6hans2jg'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/qk/0vhw5p4n2610vyrs2zqjmj2h0000gn/T/tmp6hans2jg/model.ckp

INFO:tensorflow:global_step/sec: 249.36
INFO:tensorflow:loss = 28.26818, step = 7001 (0.401 sec)
INFO:tensorflow:global_step/sec: 268.17
INFO:tensorflow:loss = 29.699017, step = 7101 (0.373 sec)
INFO:tensorflow:global_step/sec: 183.299
INFO:tensorflow:loss = 30.459818, step = 7201 (0.546 sec)
INFO:tensorflow:global_step/sec: 202.489
INFO:tensorflow:loss = 28.183102, step = 7301 (0.494 sec)
INFO:tensorflow:global_step/sec: 245.228
INFO:tensorflow:loss = 31.609444, step = 7401 (0.408 sec)
INFO:tensorflow:global_step/sec: 206.041
INFO:tensorflow:loss = 27.979221, step = 7501 (0.485 sec)
INFO:tensorflow:global_step/sec: 235.562
INFO:tensorflow:loss = 24.523727, step = 7601 (0.424 sec)
INFO:tensorflow:global_step/sec: 272.298
INFO:tensorflow:loss = 23.544832, step = 7701 (0.367 sec)
INFO:tensorflow:global_step/sec: 261.173
INFO:tensorflow:loss = 25.875946, step = 7801 (0.383 sec)
INFO:tensorflow:global_step/sec: 281.2
INFO:tensorflow:loss = 26.609013, step = 7901 (0.356 sec)
INFO:tensorflow

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x10d672c50>, 'hidden_units': [50, 20], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': <tensorflow.python.training.adam.AdamOptimizer object at 0x10d672ba8>, 'activation_fn': <function leaky_relu at 0x10f726f28>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [151]:
# PREDICT: RF for Regression
y_test_rf = estimator_rf.predict(X_test).flatten().tolist()

# PREDICT: GBR 
y_test_gbr = estimator_gbr.predict(X_test).flatten().tolist()

# PREDICT: ADA
y_test_ada = estimator_ada.predict(X_test).flatten().tolist()

# PREDICT: BAGGING
y_test_bagging = estimator_bagging.predict(X_test).flatten().tolist()

# PREDICT: KNN
y_test_knn = estimator_knn.predict(X_test).flatten().tolist()


In [152]:
new_stack = pd.DataFrame(data={'rf':y_test_rf, 'gbr':y_test_gbr,
                               'ada':y_test_ada,'bagging': y_test_bagging,
                               'true':y_test})

In [153]:
y_new_bl = list(blend_nn.predict(new_stack[['rf', 'gbr', 'ada','bagging']]))

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/qk/0vhw5p4n2610vyrs2zqjmj2h0000gn/T/tmp6hans2jg/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [154]:
model_predictions_ensemble = convertToPrediction(y_for_calculations,y_new_bl)
print(profit(actual_predictions,model_predictions_ensemble).sum())

1576722.0
